In [ ]:
import os
import pandas as pd

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

import wfdb

In [ ]:

try:
    from google.colab import drive
    drive.mount('/content/drive')
    base_directory = '/content/drive/My Drive/MSc Machine Learning/MSc Project'
    os.chdir(base_directory)
except ModuleNotFoundError:
    base_directory = ''

In [ ]:
# # Make a download directory in your current working directory
# cwd = os.getcwd()
# dl_dir = os.path.join(cwd, "mimic3wdb")
#
# Download record
# record_id = str(3141595)
# wfdb.dl_database(
#     "mimic3wdb", dl_dir=dl_dir, records=[f"{record_id[:2]}/{record_id}/{record_id}"]
# )
#
# # Display the downloaded content in the folder
# display(os.listdir(dl_dir))

In [ ]:
record_id = str(3141595)
record = wfdb.rdrecord(f"mimic3wdb/{record_id[:2]}/{record_id}/{record_id}")

In [ ]:
bvp_index = record.sig_name.index("PLETH")
bvp_signal = record.p_signal[:, bvp_index]

In [ ]:
def split_into_contiguous_segments(signal):
    """
    Signal is read from WFDB multi-segment record. Some segments don't contain signal of interest.
    We split the continuous "single" record signal into contiguous subarrays.
    :param signal:
    :return:
    """
    masked_signal = np.ma.masked_invalid(signal)
    slices = np.ma.clump_unmasked(masked_signal)
    contiguous_segments = [signal[slice] for slice in slices]
    return contiguous_segments


In [ ]:
continugous_bvp_signals = split_into_contiguous_segments(bvp_signal)

In [ ]:
signal0 = continugous_bvp_signals[0]
plt.plot(signal0)
plt.show()

In [ ]:
step = 500
view = sliding_window_view(signal0, 1000)[::step]

In [ ]:
for window in view[:20]:
    plt.plot(window)
    plt.show()
# %%
np.array_equal(view[10], signal0[10:1010])

In [ ]:
short_bvp_signal = bvp_signal[:2000]

In [ ]:
start_time = record.base_time

In [ ]:
pd_datetime = pd.to_datetime(arg=1, origin=start_time, unit="s")

In [ ]:
import datetime
from pandas.tseries.offsets import DateOffset

In [ ]:
dummy_date = datetime.date(
    year=1970, month=1, day=1
)  # I don't think the date was recorded originally
start_datetime = datetime.datetime.combine(date=dummy_date, time=start_time)

In [ ]:
timestep_second = 1 / record.fs
timestep_microsecond = int(timestep_second * 1e6)

freq = DateOffset(microseconds=timestep_microsecond)

index = pd.date_range(
    start=start_datetime, periods=len(short_bvp_signal), freq=freq, name="time"
)

In [ ]:
short_bvp_signal_df = pd.DataFrame(
    index=index, data=short_bvp_signal, columns=["physical_signal"]
)

In [ ]:
index = pd.date_range(
    start=start_datetime, periods=len(bvp_signal), freq=freq, name="time"
)
bvp_signal_df = pd.DataFrame(
    index=index, data=bvp_signal, columns=["physical_signal"]
)

In [ ]:
plt.plot(short_bvp_signal_df)
plt.ylim(0, 1)
plt.show()